# Econometría Aplicada 1 (Taller) - Examen Final
Importar librerías

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## Ejercicio 3: Índice hedónico de computadoras
Cargar datos

In [ ]:
df = pd.read_csv('../dat/chow.txt', sep='\t')

### (a) Construcción de atributos
Crear los atributos necesarios

In [ ]:
# Agregar término constante
df['CONST'] = 1

# Crear atributos logarítmicos
for col in ['RENT','MULT','ACCESS','ADD']:
    df['LN'+col] = np.log(df[col])
    
# Crear Memory Space
df['LNMEM'] = np.log(df[['WORDS','BINARY','DIGITS']].product(axis=1))

# Crear dummies por año (61, 62, ..., 65)
df['D'] = np.where(df['YEAR'].lt(61), 60, df['YEAR'])
df = pd.get_dummies(df, prefix='D', prefix_sep='_', columns=['D'], drop_first=True)

Comparación de matríces de correlación en los periodos
- [1954-1959]

In [ ]:
c1 = df.loc[df['YEAR'].between(54,59), 'LNRENT':'LNMEM'].corr()
c1.round(2)

- [1960-1965]

In [ ]:
c2 = df.loc[df['YEAR'].between(60,65), 'LNRENT':'LNMEM'].corr()
c2.round(2)

Los coeficientes de correlación entre las siguientes variables cambiaron menos de 10 puntos porcentuales entre ambos periodos:
1. `LNRENT` y `LNMEM`
1. `LNMULT` y `LNACCESS`
1. `LNMULT` y `LNADD`
1. `LNACCESS` y `LNADD`

Los demás pares de variables cambian en 10 o más puntos porcentuales.

En general, ambas matríces de correlación se parecen. Sin embargo, considero que hay diferencias importantes entre ambos periodos. En particular, hay menos pares de variables con coeficientes de correlación mayores a 0.7 en valor absoluto en el segundo periodo.

Igualmente, creo que Chow estaba en lo correcto al preocuparse por la posibilidad de un problema de multicolinealidad, pues todos los pares de variables excepto por `LNMEM` y `LNACCESS` tienen un coeficiente de correlación de almenos 0.7 en valor absoluto en el periodo previo a 1960.

Las correlaciones siguen siendo altas de 1960 en adelante. Sin embargo, los coeficientes tienden a disminuir en esta ventana de tiempo.

### (b) Regresión de Chow

In [ ]:
# Objetos para regresión
feats = ['CONST','D_61','D_62','D_63','D_64','D_65','LNMULT','LNMEM','LNACCESS']
targt = 'LNRENT'
X = df.loc[df['YEAR'].ge(60), feats]
y = df.loc[df['YEAR'].ge(60), targt]

# Modelo de Chow
m1 = sm.OLS(endog=y, exog=X)
m1_res = m1.fit()

In [ ]:
print(m1_res.summary().tables[1])
print('Observations:', int(m1_res.nobs))
print('R2:', round(m1_res.rsquared, 3))

Índice de precios usando método alternativo

In [ ]:
t = pd.concat([pd.Series([1], ['D_60']), np.exp(m1_res.params['D_61':'D_65'])]).to_frame()
t.columns=['Price Index (exp)']
t['Price Index (Chow)'] = [1,.8438,.6414,.5330,.3906,.3188]
t.round(4)

In [ ]:
plt.plot(t.index, t['Price Index (exp)'], label='Coeficientes exponenciados')
plt.plot(t.index, t['Price Index (Chow)'], label='Método Chow')
plt.xticks(t.index, labels=range(1960,1966), rotation=45, fontsize=12)
plt.xlabel('Año', fontsize=12)
plt.ylabel('Índice de precios (base 1960)', fontsize=12)
plt.legend()
plt.show()

El índice de precios usando el método de exponenciar los coeficientes de efectos fijos por año da resultados muy similares al método de Chow. Ambas series muestran la misma tendencia a la baja y las diferencias son negligibles.

### (c) Especificación alternativa
Prueba de que la especificación de Chow asume que el logaritmo `BINARY * DIGITS` y de `WORDS` tienen la misma pendiente:
$$\beta \ln(MEM) = \beta \ln(WORDS \times BINARY \times DIGITS) = \beta \ln(WORDS) + \beta \ln(BINARY \times DIGITS)$$

In [ ]:
# Separar LNMEM en dos términos independientes
df = df.assign(LNLENGTH=np.log(df['BINARY'].multiply(df['DIGITS'])),
               LNWORDS=np.log(df['WORDS']))

# Objetos para regresión
feats = ['CONST','D_61','D_62','D_63','D_64','D_65','LNMULT','LNLENGTH','LNWORDS','LNACCESS']
X = df.loc[df['YEAR'].ge(60), feats]
y = df.loc[df['YEAR'].ge(60), 'LNRENT']

# Modelo alternativo
m2 = sm.OLS(y, X)
m2_res = m2.fit()
print(m2_res.summary().tables[1])

Probar si los coeficientes son distintos entre sí:

In [ ]:
print('R2 adj de modelo 1:', round(m1_res.rsquared_adj, 4))
print('R2 adj de modelo 2:', round(m1_res.rsquared_adj, 4))

# Probar si los coeficientes son distintos
m2_res.t_test([0,0,0,0,0,0,0,1,-1,0]).summary()

Ambas especificaciones resultan en prácticamente el mismo coeficiente $R^2$ adjustado, así que la especificación no tienen un impacto sobre el desempeño predictivo del modelo. Sin embargo, los coeficientes de `LNLENGTH` y de `LNWORDS` son muy parecidos y no podemos rechazar la hipótesis nula de que tengan el mismo efecto sobre la variable dependiente.

Por cuestiones de parsimonía, prefiero el primer modelo.

### (d) Efectos fijos sin término constante
Usar todas las observaciones y crear efectos fijos por año (sin constante)

In [ ]:
# Objetos para regresión
X = pd.concat([pd.get_dummies(df['YEAR'], prefix='D'), df[['LNMULT','LNMEM','LNACCESS']]], axis=1)
y = df['LNRENT']

# Modelo
m3 = sm.OLS(y, X)
m3_res = m3.fit()
print(m3_res.summary().tables[1])

In [ ]:
t = np.exp(m3_res.params['D_54':'D_65'] - m3_res.params['D_54']).to_frame()
t.columns = ['Price Index (base 1954)']
t['Price Index (base 1965)'] = t['Price Index (base 1954)'].div(t.loc['D_65'].item())
(t * 100).round(0)

La segunda columna es igual a la primera salvo porque renormalicé la serie a 1965.

Con respecto a la serie de Triplett, mis estimaciones resultan en índices más bajos. Por ejemplo, en 1954, el precio de una computadora era 10 veces el índice de 1965. De acuerdo a los coeficientes estimados por Triplett, el precio de la misma computadora era 13.20 veces el precio de 1965.

### (e) Heteroscedasticidad

In [ ]:
# Objetos para regresión
w = np.sqrt(df.loc[df['YEAR'].ge(60), 'VOLUME'])
X = df.loc[df['YEAR'].ge(60), m1.exog_names]
y = df.loc[df['YEAR'].ge(60), 'LNRENT']

# Modelo
m4 = sm.OLS(y, X.multiply(w, axis=0), hasconst=True)
m4_res = m4.fit()
print(m4_res.summary().tables[1])

Me parece que este método no es correcto porque el libro muestra un ejemplo donde $Y_i$ es el precio promedio del modelo $i$ en la siguiente especificación:
$$Y_i = \beta_0 + \beta_1 X_{1,i} + ... + u_i$$

Nuestra especificación usa $\ln(Y_i)$ como variable dependiente, la cual no es una transformación lineal. Es decir, $\sqrt{S_m} \ \times \ln(Y_i) \neq \ln\big(Y_i \times \sqrt{S_m} \ \big)$.

Solo deberíamos de aplicar esta transformación cuando modelamos el precio de las computadoras, no el logaritmo del precio.

Como se muestra a continuación, el método que no usa este atajo coincide casi puntualmente con la estimación original salvo que los errores estándar son más grandes en la mayoría de los casos.

In [ ]:
m5 = sm.OLS(y, X, weights=w).fit()
print(m5.summary().tables[1])

## Ejercicio 4: IBM
Cargar datos

In [2]:
df = pd.read_csv('../dat/cole.txt', sep='\t')

In [4]:
df.columns

Index(['Obs', 'DEVICE', 'FCS', 'CAP', 'ACT', 'ACCT', 'ROTD', 'TR', 'ARDEN',
       'PRICE', 'SPEED', 'OLDTR', 'OLDARD', 'OLDPR', 'OLDSP', 'D72', 'D73',
       'D74', 'D75', 'D76', 'D77', 'D78', 'D79', 'D80', 'D81', 'D82', 'D83',
       'D84', 'YEAR', 'LNPRICE', 'CONST', 'LNSPEED', 'LNCAP'],
      dtype='object')

### (a) Regresión
La pregunta indica que tengo que usar los datos viejos.

In [16]:
# Nuevas columnas
df = df.assign(CONST=1,
               LNOLDPR=np.log(df['OLDPR']),
               LNPRICE=np.log(df['PRICE']),
               LNOLDSP=np.log(df['OLDSP']),
               LNSPEED=np.log(df['SPEED']),
               LNCAP=np.log(df['CAP']))

# Objetos para regresión
feats = ['CONST','D73','D74','D75','D76','D77','D78','D79','D80',
         'D81','D82','D83','D84','LNOLDSP','LNCAP']
X = df[feats]
y = df['LNOLDPR']

# Modelo
m1 = sm.OLS(y, X).fit()
print(m1.summary().tables[1])
print('R2:', m1.rsquared)

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CONST          9.4803      0.739     12.830      0.000       8.009      10.952
D73            0.0531      0.121      0.438      0.663      -0.189       0.295
D74           -0.1828      0.115     -1.588      0.116      -0.412       0.046
D75           -0.2743      0.115     -2.381      0.020      -0.504      -0.045
D76           -0.3861      0.111     -3.482      0.001      -0.607      -0.165
D77           -0.3916      0.123     -3.187      0.002      -0.636      -0.147
D78           -0.5500      0.169     -3.263      0.002      -0.886      -0.214
D79           -0.7449      0.169     -4.419      0.000      -1.081      -0.409
D80           -0.9365      0.162     -5.779      0.000      -1.259      -0.614
D81           -0.9455      0.161     -5.889      0.000      -1.265      -0.626
D82           -0.9334      0.169     -5.513      0.0

### (b) Misma regresión usando nuevos datos

In [17]:
feats = ['CONST','D73','D74','D75','D76','D77','D78','D79','D80',
         'D81','D82','D83','D84','LNSPEED','LNCAP']
X = df[feats]
y = df['LNPRICE']

# Modelo nuevo
m2 = sm.OLS(y, X).fit()
print(m2.summary().tables[1])
print('R2:', m2.rsquared)

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CONST          9.4283      0.757     12.459      0.000       7.921      10.936
D73            0.0160      0.121      0.132      0.896      -0.225       0.257
D74           -0.2177      0.115     -1.891      0.062      -0.447       0.012
D75           -0.3092      0.115     -2.685      0.009      -0.539      -0.080
D76           -0.4173      0.111     -3.764      0.000      -0.638      -0.197
D77           -0.4167      0.123     -3.387      0.001      -0.662      -0.172
D78           -0.5740      0.169     -3.399      0.001      -0.910      -0.238
D79           -0.7689      0.169     -4.553      0.000      -1.105      -0.433
D80           -0.9602      0.162     -5.915      0.000      -1.283      -0.637
D81           -0.9670      0.161     -6.014      0.000      -1.287      -0.647
D82           -0.9537      0.170     -5.626      0.0

Todos los coeficientes se mantienen cercanos a lo que solían ser. En particular, los coeficientes estimados en la vieja y nueva regresión resultan en que las mismas variables tienen un efecto estadísticamente significativo.

El efecto marginal de `LNSPEED` bajo, pero no mucho.

### (c) Homogeneidad
$H_0$: $\beta_{lnspeed} + \beta_{lncap} = 1$

$H_1$: $\beta_{lnspeed} + \beta_{lncap} \neq 1$

In [34]:
m2.t_test(np.zeros(len(m2.params)-2).astype(int).tolist() + [1, 1])

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.8497      0.074     11.430      0.000       0.702       0.998

Podemos rechazar la hipótesis nula de que los coeficientes sean iguales a uno. Esto significa que al multiplicar ambas variables por $\lambda$, el precio se multiplicaría por un factor menor a $\lambda$.

### (d) Efectos anuales constantes
Si reemplazamos los efectos fijos anuales por una sola columna de tiempo, estamos asumiendo que el paso del tiempo tiene rendimientos constantes sobre el logaritmo del precio porque estamos estimando un solo coeficiente.

In [50]:
# Nueva variable de rendimientos constantes
df['TIME'] = df['YEAR'] - 1971

# Objetos para regresión
X = df[['CONST','TIME','LNSPEED','LNCAP']]
y = df['LNPRICE']

# Modelo
m3 = sm.OLS(y, X).fit()
print(m3.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
CONST          9.7808      0.639     15.298      0.000       8.510      11.052
TIME          -0.1052      0.012     -8.424      0.000      -0.130      -0.080
LNSPEED        0.4343      0.109      3.988      0.000       0.218       0.651
LNCAP          0.4428      0.066      6.745      0.000       0.312       0.573


Haré la prueba $F$ de que todos los coeficientes sean iguales.

In [61]:
np.roll([0,1-b,0,0,0,0,0,0,0,0,0,0,0,0,0], 1)

array([0.        , 0.        , 1.10515499, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [73]:
# Efecto marginal anual
b = m3.params['TIME']

# Matriz de restricciones
L = np.matrix([[0,1-b,0,0,0,0,0,0,0,0,0,0,0,0,0],
               [0,0,1-b,0,0,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,1-b,0,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,0,1-b,0,0,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,1-b,0,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,1-b,0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,0,1-b,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,0,0,1-b,0,0,0,0,0,0],
               [0,0,0,0,0,0,0,0,0,1-b,0,0,0,0,0],
               [0,0,0,0,0,0,0,0,0,0,1-b,0,0,0,0],
               [0,0,0,0,0,0,0,0,0,0,0,1-b,0,0,0],
               [0,0,0,0,0,0,0,0,0,0,0,0,1-b,0,0]])

# Prueba
f = m2.f_test(L)
print('F-stat:', round(f.statistic, 4)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[6.1990736]]), p=1.7806867071957074e-07, df_denom=76, df_num=12>

In [66]:
f.summary()

'<F test: F=array([[6.1990736]]), p=1.7806867071957074e-07, df_denom=76, df_num=12>'

## Ejercicio 5: Estabilidad de precios hedónicos

## Ejercicio 6: Índice encadenado

## Ejercicio 7: Box-Cox